In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import random
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import recall_score, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
#from userclass import User
import json

In [2]:
tagsFile = "tags"
recipesFile = "recipes"
nutritionsFile = "nutritions"
ingredientsFile = "ingredients"
ingredientsdf = pd.read_csv(f"/data/foodboost/{ingredientsFile}.csv", index_col=0)
tagsdf = pd.read_csv(f"/data/foodboost/{tagsFile}.csv", index_col=0)
recipesdf = pd.read_csv(f"/data/foodboost/{recipesFile}.csv", index_col=0)

In [3]:
#INGREDIENTEN
def recepten_bij_ingredient(ingredient):
    a = ingredientsdf.loc[ingredientsdf['ingredient'] == ingredient].recipe.to_list()
    return a
def ingredienten_bij_recept(recept):
    b = ingredientsdf.loc[ingredientsdf['recipe'] == recept].ingredient.unique().tolist()
    return b

#TAGS
def recepten_bij_tag(tag):
    a = tagsdf.loc[tagsdf['tag'] == tag].recipe.to_list()
    return a

def tags_bij_recept(gerecht):
    b = tagsdf.loc[tagsdf['recipe'] == gerecht].tag.unique()
    return b

In [4]:
diner_recepten = list(set(recepten_bij_tag('diner') + recepten_bij_tag('hoofdgerecht')))

In [5]:
TotalTrainMatrix = pd.read_csv('trainMatrix.csv')
TotalValidateMatrix = pd.read_csv('validateMatrix.csv')
TotalTestMatrix = pd.read_csv('testMatrix.csv')

In [6]:
TotalTrainMatrix

,Unnamed: 0,koelverse sereh citroengras,kokosmelk,water,laurierblad,steranijs,ribrunderlap,kardemompeulen,korianderzaad,komijnzaad,...,2-prei a la creme deelblokjes,2-mix voor ovenpasta tomaat-mozzarella,2-pastasaus zongedroogde tomaat &amp; oregano,2-boomgaardsap peer,2-zalmreepjes,2-gehakt half-om-half,2-rodekool met appel,y,Randomgerecht,one out
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,NaN,['Rijkgevulde groente-enchilada’s met mais &am...
1,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,Andijviestamppot met gebakken salami,['Rijkgevulde groente-enchilada’s met mais &am...
2,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,NaN,"['Tortilla met koolsalade, bonen en vissticks']"
3,3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,Groentebolognese met Parmezaanse kaas,"['Tortilla met koolsalade, bonen en vissticks']"
4,4,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,NaN,['Mexicaanse enchilada’s met gehakt &amp; toma...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,23995,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,Aziatische groenteroerbak met seitan en ketjap,['Snel\xadle ham\xadprei\xadscho\xadtel']
23996,23996,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,NaN,['Hutspot met vegaballetjes']
23997,23997,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,Kip uit de braadzak,['Hutspot met vegaballetjes']
23998,23998,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,NaN,"[""Kip 'tikka massala' met ovengroente en yoghu..."


In [7]:
ingredientsdf = ingredientsdf[ingredientsdf['recipe'].isin(diner_recepten)]

In [8]:
meeste_ingredienten = ingredientsdf.groupby(['ingredient']).recipe.count().to_frame().sort_values(['recipe'], ascending = False).iloc[0:500]

In [9]:
meeste_ingredienten

,recipe
ingredient,
knoflook,1446
traditionele olijfolie,1088
milde olijfolie,676
ongezouten roomboter,587
middelgroot ei,583
...,...
Saitaku rijstazijn,14
ribkarbonade,14
chilisaus,13


In [10]:
lijst500 = meeste_ingredienten.index.values.tolist()
lijst500_2 = ['2-' + columnName for columnName in lijst500]
lijst1000 = lijst500+lijst500_2
#lijst1000.append('y')

In [11]:
X_train = TotalTrainMatrix[TotalTrainMatrix.columns.intersection(lijst1000)]

In [12]:
X_train

,koelverse sereh citroengras,kokosmelk,water,steranijs,komijnzaad,kruidnagel,chilivlokken,gemalen kaneel,sjalot,knoflook,...,2-pappardelle,2-wilde spinazie,2-champignonroerbakmix,2-lamsworstjes,2-koolzaadolie,2-boter,2-verse pompoen,2-wokolie,2-aardappelen,2-eieren
0,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
23996,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
23997,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
23998,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [20]:
y_train = TotalTrainMatrix['y'].to_frame()

In [21]:
X_test = TotalTestMatrix[TotalTestMatrix.columns.intersection(lijst1000)]

In [22]:
model = KNeighborsClassifier()
model2 = tree.DecisionTreeClassifier()

In [23]:
model.fit(X_train, y_train.values.ravel())
model2.fit(X_train, y_train.values.ravel())

DecisionTreeClassifier()

In [24]:
y_pred = model.predict(X_test)
y_pred2 = model2.predict(X_test)

In [25]:
y_test = y_train[0:1600]

In [26]:
print("Accuracy Score KNN: ", accuracy_score(y_test, y_pred))
print("Accuracy Score DecisionTree: ", accuracy_score(y_test, y_pred2))


Accuracy Score KNN:  0.5025
Accuracy Score DecisionTree:  0.48875
